In [5]:
from data_parser2 import DataParser
from datetime import datetime
import pandas as pd

In [16]:
dp = DataParser(filename='data/GBPUSD.csv')

In [17]:
start_datetime = datetime.strptime('20101001 223600', '%Y%m%d %H%M%S')
end_datetime = datetime.strptime('20120228 183600', '%Y%m%d %H%M%S')

In [18]:
dp.filter_by_date(start_datetime, end_datetime)
dp.sample_by_time('40min', 2160000000000)
Xn,yn = dp.get_close_price_numpy_array(0, 21, 21, 30,36)

In [19]:
Xn1 = dp.tranform_X_to_returns(Xn)

In [23]:
Xn2 = dp.transform_X_with_IWMA(Xn1)

In [24]:
X_norm, y_norm = dp.normalize_data(Xn2, yn)

In [25]:
print(X_norm.shape)
print(y_norm.shape)

(326, 21)
(326,)
